In [1]:
from main import RF
from mmdit import MMDiT_for_IN1K
import torch



rf = RF(
    MMDiT_for_IN1K(
        in_channels=4,
        out_channels=4,
        dim=1024,
        n_layers=6,
        n_heads=8,
    ),
    True,
).cuda()

rf.load_state_dict(torch.load('ckpt/model_44001/ema2.pt'))


    

/home/host/py39cuda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-04 18:04:25,123] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


<All keys matched successfully>

In [2]:
init_noise = torch.randn(16, 4, 32, 32).cuda()
conds = [933, 849, 94, 230] * 4
conds = torch.tensor(conds).cuda()
images = rf.sample(init_noise, conds, null_cond = torch.tensor([1000] * 16).cuda(), sample_steps = 50, cfg = 4)

In [4]:
from diffusers.image_processor import VaeImageProcessor
from diffusers.models import AutoencoderKL

vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae").to("cuda:0")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [6]:
for i, labidx in enumerate(conds):
    x = vae.decode(images[-1][i : i + 1].cuda()/0.13025).sample
    img = VaeImageProcessor().postprocess(
        image=x.detach(), do_denormalize=[True, True]
    )[0]
    img.save(f"sample_{i}.png")